# Weather Data

## Loading Env Variables

In [25]:
from configparser import ConfigParser
job_name = 'influx viz'
path = '/home/armando/Github Repos/weatherAPI_data_eng/src/'

config_object = ConfigParser(interpolation = None)
config_object.read(path + 'config.ini')

influx_db = config_object['DATABASE']    
my_token = influx_db['token']
my_org = influx_db['organization']
my_agg = influx_db['agg_bucket']

## Loading the aggregations

In [156]:
from influxdb_client import InfluxDBClient

with InfluxDBClient(url='https://us-east-1-1.aws.cloud2.influxdata.com', token=my_token, org=my_org) as client:
        # start 0, brings all available data
        # pivot, structures the influx tables into a single df
        # measurement filter, so that when I add new measures this job isn't affected    
        q = f""" from(bucket:"{my_agg}") 
                |> range(start:0)
                |> filter(fn: (r) => r["_measurement"] == "weather_aggregations")
                |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value") 
                |> group()""" #Otherwise it outputs several tables
        df_query = client.query_api().query_data_frame(q)
        df_query.drop(["table", "result", "_start", "_stop", "_measurement"], inplace=True, axis=1)
        df_query = df_query.sort_values(by=['_time'], ascending=False)
        client.close()

In [ ]:

timezones = {"Monterrey":"America/Mexico_City", 
             "Boston": "America/NewYork", 
             "Saltillo":"America/Monterrey", 
             "Guadalajara":"America/Monterrey", 
             "Mexico City":"America/Monterrey"}

city = "Monterrey"
df_query.set_index("_time", inplace=True)
df_query.sort_index(inplace = True)
df_query.index = df_query.index.tz_convert(timezones[city])

print(df_query[(df_query["city"] == "Boston") & (df_query["window_time"] == "12H") & (df_query["agg"] == "mean")].head())

### Seaborn / Streamlit